In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket1 = sagemaker_session.default_bucket()
print(bucket1)

sagemaker-ap-northeast-2-137645371426


In [3]:
from datetime import datetime
from HRC_0819_predict2 import get_matching_s3_keys
bucket = 'hrms-aggr-train'
prefix = 'regular'
suffix = 'json'
y = datetime.now().strftime(f'yyyy=%Y')
m = datetime.now().strftime(f'mm=%m')
d = datetime.now().strftime(f'dd=%d')
data_dir = datetime.now().strftime('%Y-%m-%d')
#######Fix Data Location#############
d = 'dd=14'
######################
#print(data_dir)
s3_nowdir=f'{prefix}/{y}/{m}/{d}'
#print(s3_nowdir)
#keys = [k for k in get_matching_s3_keys(bucket, s3_nowdir, suffix)]
#print(keys[0])

In [5]:
inputs = sagemaker_session.upload_data(path=f'data/', bucket=bucket, key_prefix=s3_nowdir)
print(inputs)
type(inputs)

s3://hrms-aggr-train/regular/yyyy=2020/mm=08/dd=14


str

In [3]:
!pwd

/home/ec2-user/SageMaker/AmazonSageMaker-sagemaker-seoul/hrms


In [5]:
import pygments
!pygmentize ./HRC_0819_predict2.py


#!/usr/bin/env python
# coding: utf-8

# In[2]:


import argparse
import json
import logging
import os

import sys
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data.distributed
import torch.utils.data
from torchvision import datasets, transforms
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import pandas as pd
import json
import collections
import numpy as np
import boto3 
from sklearn.metrics import mean_squared_error as mse
from six import BytesIO

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(6, 5),
            nn.BatchNorm1d(5),
            nn.LeakyReLU(),
            nn.Linear(5, 4),
            nn.BatchNorm1d(4),
            nn.LeakyR

In [23]:
# ipython nbconvert --to python *.ipynb
import pandas as pd
s3_output_location = 's3://{}'.format('hrms-model')
estimator = PyTorch(entry_point='HRC_0819_predict2.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.c5.4xlarge',
                    output_path=s3_output_location,
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo',
                        'batch-size': 64,
                        'test-batch-size': 64,
                    })

estimator.fit({'training': inputs})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-26 04:23:58 Starting - Starting the training job...
2020-08-26 04:24:01 Starting - Launching requested ML instances......
2020-08-26 04:25:09 Starting - Preparing the instances for training...
2020-08-26 04:25:49 Downloading - Downloading input data...
2020-08-26 04:26:28 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-26 04:26:41,803 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-26 04:26:41,806 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-26 04:26:41,814 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-26 04:26:43,229 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-26 04:26:43,480 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020

In [24]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c5.4xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

In [86]:
#TEST##
output = predictor.predict('data/2020-08-18/train.jsonl')
print(output)

algo-1-8y61r_1  | 2020-08-24 01:36:35,173 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 1
algo-1-8y61r_1  | 2020-08-24 01:36:35,174 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:49650 "POST /invocations HTTP/1.1" 500 5


ValueError: Cannot load file containing pickled data when allow_pickle=False

In [19]:
from HRC_0819_predict2 import read_data
data, timestamp, robot_name  = read_data()

In [20]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()
data= scaler.fit_transform(data).astype(np.float32)
print(data)

[[ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.          0.          0.          0.14744195  0.        ]
 [ 0.          0.        

In [21]:
output = predictor.predict(data)
print(output)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "'bytes' object has no attribute 'read'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_inference/transformer.py", line 125, in transform
    result = self._transform_fn(self._model, input_data, content_type, accept)
  File "/opt/conda/lib/python3.6/site-packages/sagemaker_inference/transformer.py", line 216, in _default_transform_fn
    result = self._output_fn(prediction, accept)
  File "/opt/ml/model/code/HRC_0819_predict2.py", line 351, in output_fn
    return buffer.getvalue().read().decode(), 'application/x-npy'
AttributeError: 'bytes' object has no attribute 'read'
". See https://ap-northeast-2.console.aws.amazon.com/cloudwatch/home?region=ap-northeast-2#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-training-2020-08-26-02-49-39-489 in account 137645371426 for more information.

In [22]:
data.dtype
type(data)

numpy.ndarray

In [93]:
output.dtype
type(output)

numpy.ndarray

In [92]:
data.type

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [20]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.
